In [1]:
# results directories:
# /home/jxm3/random/interpretable-autoprompting/results/intermediate_results_from_chandan/anli
# /home/jxm3/random/interpretable-autoprompting/results/intermediate_results_from_chandan/math

%load_ext autoreload
%autoreload 2

In [4]:
import analyze_utils

In [53]:
dir_single_query = '/home/jxm3/random/interpretable-autoprompting/results/intermediate_results_from_chandan/math'
r = analyze_utils.load_results_and_cache_json(dir_single_query, save_file='r.pkl')

r['use_single_query'] = r['single_query']
r['final_answer_pos_initial_token']
r = analyze_utils.postprocess_results(r)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 445.07it/s]


In [62]:
r[(~r['single_query'].astype(bool)) & (r['do_reranking'].astype(bool))][['checkpoint', 'task_name']].value_counts()

checkpoint               task_name    
EleutherAI/gpt-neo-2.7B  add_two          128
                         multiply_two     128
gpt2-xl                  add_two          128
                         divide_two       128
                         double_one       128
                         exp_one          128
                         fibonacci_one    128
                         first_two        128
                         max_two          128
                         multiply_two     128
                         square_one       128
                         subtract_two     128
dtype: int64

In [21]:
rt['use_single_query'].value_counts()

Avg suffix      5376
Single-query    3840
Name: use_single_query, dtype: int64

In [28]:
table = (rt
    # mean over templates, task_name)
    .groupby(['checkpoint', 'n_shots', 'do_reranking', 'use_single_query', 'task_name'])[[metric_key]]
    .mean()
    .reset_index()
)
table.head(n=1000)

,checkpoint,n_shots,do_reranking,use_single_query,task_name,final_answer_pos_initial_token
0,EleutherAI/gpt-j-6B,1,0,Avg suffix,add_two,2.0
1,EleutherAI/gpt-j-6B,1,0,Avg suffix,divide_two,2.0
2,EleutherAI/gpt-j-6B,1,0,Avg suffix,double_one,NaN
3,EleutherAI/gpt-j-6B,1,0,Avg suffix,exp_one,14.0
4,EleutherAI/gpt-j-6B,1,0,Avg suffix,fibonacci_one,113.0
...,...,...,...,...,...,...
67,gpt2-xl,1,1,Avg suffix,first_two,54.0
68,gpt2-xl,1,1,Avg suffix,max_two,NaN
69,gpt2-xl,1,1,Avg suffix,multiply_two,1.0
70,gpt2-xl,1,1,Avg suffix,square_one,19.0


In [48]:
rt[rt['task_name'] == 'square_one'].groupby('checkpoint').mean()[[c for c in r.columns if 'Recall' in c]]

,Recall @ 3 suffixes,Recall @ 5 suffixes,Recall @ 10 suffixes,Recall @ 15 suffixes,Recall @ 20 suffixes,Recall @ 25 suffixes,Recall @ 30 suffixes,Recall @ 40 suffixes,Recall @ 50 suffixes,Recall @ 75 suffixes,Recall @ 100 suffixes,Recall @ 150 suffixes,Recall @ 200 suffixes
checkpoint,,,,,,,,,,,,,
EleutherAI/gpt-j-6B,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
EleutherAI/gpt-neo-2.7B,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
gpt2-xl,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [32]:
rt_model.single_query.value_counts()

0    1280
1    1280
Name: single_query, dtype: int64

In [42]:
for model in rt['checkpoint'].unique():
    print(model)
    rt_model = rt[(rt['checkpoint'] == model) & (rt['single_query'].astype(bool))]
    table = (rt_model
        # mean over templates, task_name)
        .groupby(['checkpoint', 'n_shots', 'do_reranking', 'single_query', 'task_name'])[[metric_key]]
        .mean()
        .reset_index()
    )
    print(table.head())
    print(len(table), '\n'*3)


gpt2-xl
  checkpoint  n_shots  do_reranking  single_query      task_name  \
0    gpt2-xl        1             0             1        add_two   
1    gpt2-xl        1             0             1     divide_two   
2    gpt2-xl        1             0             1     double_one   
3    gpt2-xl        1             0             1        exp_one   
4    gpt2-xl        1             0             1  fibonacci_one   

   final_answer_pos_initial_token  
0                             2.0  
1                             2.0  
2                           125.0  
3                            15.0  
4                           116.0  
10 



EleutherAI/gpt-neo-2.7B
                checkpoint  n_shots  do_reranking  single_query  \
0  EleutherAI/gpt-neo-2.7B        1             0             1   
1  EleutherAI/gpt-neo-2.7B        1             0             1   
2  EleutherAI/gpt-neo-2.7B        1             0             1   
3  EleutherAI/gpt-neo-2.7B        1             0             1   
4

In [24]:
# rt = r[r.task_name.isin(['add_two', 'multiply_two', 'subtract_two'])]
import matplotlib.pyplot as plt
rt = r
for metric_key in ['final_answer_pos_initial_token',
                   f'Recall @ {3} suffixes',
                #    f'Recall @ {10} suffixes', 
                #    f'Recall @ {50} suffixes', 
                #    f'Recall @ {100} suffixes', 
                   ]:
    table = (rt
        # mean over templates, task_name)
        .groupby(['checkpoint', 'n_shots', 'do_reranking', 'use_single_query'])[[metric_key]]
        .mean()
        .reset_index()
    )
    analyze_utils.plot_tab(tab, metric_key=metric_key, title='Math datasets')

    # if metric_key == 'final_answer_pos_initial_token':
      # plt.savefig('../results/prompt_gen/math_barplot.pdf', bbox_inches='tight')
    plt.show()


ValueError: Could not interpret input 'final_answer_pos_initial_token'